In [1]:
%%html
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [2]:
import random
from IPython.display import Image

# Configuration

In [3]:
%%javascript
require(['base/js/utils'],
function(utils) {
   utils.load_extensions('calico-spell-check', 'calico-document-tools', 'calico-cell-tools');
});

<IPython.core.display.Javascript object>

In [4]:
# Reveal.js
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'theme': 'white',
        'transition': 'none',
        'controls': 'true',
        'progress': 'true',
})

{'theme': 'white',
 'transition': 'none',
 'controls': 'true',
 'progress': 'true'}

In [5]:
%load_ext tikzmagic

In [6]:
%%html
<style>
.red { color: #E41A1C; }
.orange { color: #FF7F00 }
.yellow { color: #FFC020 }         
.green { color: #4DAF4A }                  
.blue { color: #377EB8; }
.purple { color: #984EA3 }       
       
h1 {
    color: #377EB8;
}
       
ctb_global_show div.ctb_hideshow.ctb_show {
    display: inline;
} 
         
div.tabContent {
    padding: 0px;
    background: #ffffff;     
    border: 0px;                        
}  
         
.left {
    float: left;
    width: 50%;
    vertical-align: text-top;
}

.right {
    margin-left: 50%;
    vertical-align: text-top;                            
}    
               
.small {         
    zoom: 0.9;
    -ms-zoom: 0.9;
    -webkit-zoom: 0.9;
    -moz-transform:  scale(0.9,0.9);
    -moz-transform-origin: left center;  
}          
         
.verysmall {         
    zoom: 0.75;
    -ms-zoom: 0.75;
    -webkit-zoom: 0.75;
    -moz-transform:  scale(0.75,0.75);
    -moz-transform-origin: left center;  
}         
   
        
.tiny {         
    zoom: 0.6;
    -ms-zoom: 0.6;
    -webkit-zoom: 0.6;
    -moz-transform:  scale(0.6,0.6);
    -moz-transform-origin: left center;  
}         
         
         
.rendered_html blockquote {
    border-left-width: 0px;
    padding: 15px;
    margin: 0px;    
    width: 100%;                            
}         
         
.rendered_html th {
    padding: 0.5em;  
    border: 0px;                            
}         
         
.rendered_html td {
    padding: 0.25em;
    border: 0px;                                                        
}            
         
.rise-enabled .reveal .progress span {
    background: #377EB8;
}     
         
.present .top {
    position: fixed !important;
    top: 0 !important;                                   
}                  
    
.present .rendered_html * + p, .present .rendered_html p, .present .rendered_html * + br, .present .rendered_html br {
    margin: 0.5em 0;                            
}  
         
.present tr, .present td {
    border: 0px;
    padding: 0.35em;                            
}      
         
.present th {
    border: 1px;
}
         
present .prompt {
    min-width: 0px !important;
    transition-duration: 0s !important;
}     
         
.prompt {
    min-width: 0px !important;
    transition-duration: 0s !important;                            
}         
         
.rise-enabled .cell li {
    line-height: 135%;
}
         
</style>

# Recurrent Neural Networks


* Recurrent Neural Network (RNN) Language Models
* Training Problems and Solutions
  - Vanishing and Exploding Gradients
  - Long Short-Term Memory (LSTM) Networks
* RNN applications / Flavours of RNNs
* Variations (characters, bi-directional RNNs)


# Reminder: Language Models (LM)

A LM computes a **probability** for a **sequence of words**

$$p(\langle w_{1}, \ldots, w_{d} \rangle)$$

Useful in a myriad of NLP tasks involving text generation, e.g.
- Machine Translation,
- Speech Recognition, 
- Summarisation.. 

$$
\begin{aligned}
p(\langle \text{Statistical}, \text{Natural}, \text{Language}, \text{Processing} \rangle) > \\
p(\langle \text{Statistical}, \text{Language}, \text{Natural}, \text{Processing} \rangle)
\end{aligned}
$$

# Reminder: $n$-Gram Language Models

In *$n$-gram language models*, the probability $p(w_{1}, \ldots, w_{d}) = \prod_{i=1}^{d} p(w_{i} \mid w_{1}, \ldots, w_{i - 1})$ is **approximated** as:

$$
\begin{aligned}
p(w_{1}, \ldots, w_{d}) & \approx \prod_{i=1}^{d} p(w_{i} \mid w_{i - (n - 1)}, \ldots, w_{i - 1}) \\
& \approx \prod_{i=1}^{d} \frac{\text{count}(w_{i - (n - 1)}, \ldots, w_{i})}{\text{count}(w_{i - (n - 1)}, \ldots, w_{i - 1})}
\end{aligned}
$$

Example with a **bigram** ($n = 2$) **language model**:

$$
\begin{aligned}
 p(\langle \text{Natural}, & \text{Language}, \text{Processing} \rangle) \approx \\
 & p(\text{Natural}){}\cdot{}p(\text{Language} \mid \text{Natural}) \cdot p(\text{Processing} \mid \text{Language})
\end{aligned}
$$

### Disadvantages of $n$-Gram Language Models

* Limited and small context window
* No generalization to similar words and contexts (essentially one-hot featurisation)

How can we address these shortcomings?

# Reminder: Feed-forward Neural Networks
<center><img src="../../../../assets/img/mlp.svg"></center>

But how do we use a neural network to encode a **sequence**?

# Recurrent Neural Networks

* RNNs share the weights at each time step
* The output $y_{t}$ at time $t$ depends on all previous words: $w_{t}, w_{t - 1}, \ldots, w_{1}$
* Size scales with **number of words**, not **sequence length**!

<center><img src="../img/rnn_unrolled.png" width=50%></center>

<div class="left"><div><div class="small">
<div style="margin-bottom: 00%;"></div>
\begin{align}
\mathbf{h}_t &= f_{\theta}(\mathbf{x}_{t}, \mathbf{h}_{t - 1}) \\
    f_{\theta} \; & \text{is a } \textbf{transition function} \text { with parameters } \theta\\
    \theta \; & \text{can be } \textbf{learned from data}\\
\end{align}
</div></div></div>

# Recurrent Neural Network Language Model

<center><img src="../img/rnnlm.png"></center>

# A Recurrent Neural Network LM

Consider the following sentence:

$$\langle w_{1}, \ldots, w_{t - 1}, w_{t}, w_{t + 1}, \ldots, w_{d})$$

First apply word embedding (e.g., [word2vec](dl-representations_simple.ipynb)):
$$
\begin{aligned}
 \mathbf{x}_{1} & = \text{encode}(w_{t}) \in \mathbb{R}^{d_{e}}\\
\end{aligned}
$$

At each single time step $t$, the hidden state $\mathbf{h}_t$ is given by:

$$
\begin{aligned}
\mathbf{h}_t & = \sigma(\mathbf{W}^h \mathbf{h}_{t-1}+ \mathbf{W}^x \mathbf{x}_t) \in \mathbb{R}^{d_{h}}\\
\end{aligned}
$$
Input matrix: $\mathbf{W}^x \in \mathbb{R}^{d_{h} \times d_{x}}$

Transition matrix: $\mathbf{W}^h \in \mathbb{R}^{d_{h} \times d_{h}}$

The output $\hat{\mathbf{y}}_t \in [0, 1]^{|V|}$, a **probability distribution** over words in $V$, is given by:

$$
\begin{aligned}
\hat{\mathbf{y}}_{t} & = \text{softmax}(\mathbf{W}^o \mathbf{h}_{t}) \\
\end{aligned}
$$

Output matrix: $\mathbf{W}^o \in \mathbb{R}^{|V| \times d_{h}}$

and softmax is defined as:
$$
\text{softmax}(z) = \frac{1}{\sum_{i=1}^{d_z} e^{z_i}} \left(e^{z_1}, e^{z_2},\ldots,e^{z_{d_z}}\right)
$$

The probability that the $t$-th word in the sequence is $w_{j}$ is given by:

$$p(w_{j} \mid w_{t}, \ldots, w_{1}) = \hat{\mathbf{y}}_{t, j}$$

# Example

Consider the word sequence $\text{encode}(\text{Natural}, \text{Language}, \text{Processing}) \rightarrow (\mathbf{x}_{1}, \mathbf{x}_{2}, \mathbf{x}_{3})$

Reminder: $\mathbf{h}_t = \sigma(\mathbf{W}^h \mathbf{h}_{t-1}+ \mathbf{W}^x \mathbf{x}_t + \mathbf{b})$

$$
\begin{aligned}
 \mathbf{h}_1 = \sigma(\mathbf{W}^h \mathbf{h}_{0} + \mathbf{W}^x \mathbf{x}_1) &\;& \hat{\mathbf{y}}_{1} = \text{softmax}(\mathbf{W}^o \mathbf{h}_{1}) \\
 \mathbf{h}_2 = \sigma(\mathbf{W}^h \mathbf{h}_{1} + \mathbf{W}^x \mathbf{x}_2) &\;& \hat{\mathbf{y}}_{2} = \text{softmax}(\mathbf{W}^o \mathbf{h}_{2}) \\
 \mathbf{h}_3 = \sigma(\mathbf{W}^h \mathbf{h}_{2} + \mathbf{W}^x \mathbf{x}_3) &\;& \hat{\mathbf{y}}_{3} = \text{softmax}(\mathbf{W}^o \mathbf{h}_{3}) \\
\end{aligned}
$$

$$p(\text{Natural}, \text{Language}, \text{Processing}) = \hat{\mathbf{y}}_{1, [\text{Natural}]} \; \hat{\mathbf{y}}_{2, [\text{Language}]} \; \hat{\mathbf{y}}_{3, [\text{Processing}]}$$

- Initial state: $\mathbf{h}_{0} \in \mathbb{R}^{d_{h}}$
- Input matrix: $\mathbf{W}^x \in \mathbb{R}^{d_{h} \times d_{x}}$, Transition matrix: $\mathbf{W}^h \in \mathbb{R}^{d_{h} \times d_{h}}$, Output matrix: $\mathbf{W}^o \in \mathbb{R}^{|V| \times d_{h}}$

<center><img src="../img/quiz_time.png"></center>

# [tinyurl.com/diku-nlp-unk](https://tinyurl.com/diku-nlp-unk)
([Responses](https://docs.google.com/forms/d/1Ze1pUQkraElKVcMU8zsNNe7GFmRQluffFi-J_aIvvCk/edit#responses))

# Objective Function

Recall that $\hat{\mathbf{y}}_{t} \in \mathbb{R}^{|V|}$ is a probability distribution over the vocabulary $V$.

We can train a RNN by minimizing the **cross-entropy loss**, predicting **words** instead of classes:

$$
\begin{aligned}
J_{t} = - \sum_{i = 1}^{|V|} \mathbf{y}_{t, i} \log \hat{\mathbf{y}}_{t, i}, \quad \text{where} \quad \mathbf{y}_{t, i} = \left\{\begin{array}{ll}1 \; \text{if the $t$-th word is $w_{i}$,}\\0 \, \text{otherwise.}\end{array} \right.
\end{aligned}
$$

# Problem - Training RNNs is Hard

- **Vanishing** and **exploding** gradients [<span class=blue>Pascanu et al. 2013</span>].

Why? Multiply the same matrix $\mathbf{W}^{h}$ at each time step during forward propagation. The norm of the gradient might either tend to 0 (**vanish**) or be too large (**explode**).

<center>
<img src="rnn-figures/error_surface.png" width="40%"/>
</center>

# Related Problem - Long-Term Dependencies

Words from time steps far away are hardly considered when training to predict the next word.

Example:
- John walked to the hallway.
- Mary walked in too.
- Daniel moved to the garden.
- John said "Hi" to \_\_\_\_.

or

- When I moved to France, I quickly ran into difficulties communicating, because I don't speak any \_\_\_\_.

# Vanishing/Exploding Gradients - Solutions

Several solutions in the literature:

- Bound the gradient to a threshold (**Gradient Clipping**)<br>[<span class=blue>Pascanu et al. 2013</span>]

- Use $\text{ReLU}(x) = \max(0, x)$ (**Re**ctified **L**inear **U**nits) or similar non-linearities instead of $\text{sigmoid}(x)$ or $\text{tanh}(x)$<br>[<span class=blue>Glorot et al. 2011</span>].

- Clever initialization of the transition matrix ($\mathbf{W}^h = \mathbf{I}$)<br>[<span class=blue>Socher et al. 2013</span>, <span class=blue>Le et al. 2015</span>].

- Use different recurrent models that favour backpropagation<br>LSTM [<span class=blue>Hochreiter et al. 1997</span>], GRU [<span class=blue>Chung et al. 2014</span>].

# Long Short-Term Memory (LSTM) Networks

- Can adaptively learn what to **keep** (store) into memory (gate $\mathbf{i}_{t}$), **forget** (gate $\mathbf{f}_{t}$) and **output** (gate $\mathbf{o}_{t}$)

<center><img src="../img/rnn_overview.png" width="60%"></center>

<center><img src="../img/lstm_internal-v2.png" width="60%"></center>

# Long Short-Term Memory (LSTM) Networks

- Can adaptively learn what to **keep** (store) into memory (gate $\mathbf{i}_{t}$), **forget** (gate $\mathbf{f}_{t}$) and **output** (gate $\mathbf{o}_{t}$)
- Memory from current time step: $\mathbf{c}_{t}$; hidden state from previous time step: $h_{t-1}$
- Element-wise multiplication: <span style="background-color: black; color: white">x</span>; element-wise summation: <span style="background-color: black; color: white">+</span>
<center><img src="../img/lstm_internal-v2.png" width=70%></center>

<center><img src="../img/lstm_equations-v2.png", width=600></center>

## Three Flavours of RNNs in NLP ##

Encoder-only:
* Many-to-one (sequence classification)
* One-to-one (sequence tagging/labeling)

Encoder-decoder:
* Many-to-many (sequence-to-sequence or seq2seq)

## Many-to-one ##

<center><img src="../img/many_to_one.png"></center>

## One-to-one ##
<center><img src="../img/one_to_one2.png"></center>


## Many-to-many ##
<center><img src="../img/many_to_many.png" width="80%"></center>

# Sequence-to-Sequence Models

Recurrent Neural Networks are extremely powerful and flexible
- They can also learn to **generate** sequences

Seq2Seq models are composed by:
- **Encoder** - Gets the input and outputs $\mathbf{v} \in \mathbb{R}^{d}$
- **Decoder** - Gets $\mathbf{v}$ and generates the output sequence

Seq2Seq models are widely popular in e.g.:
- *Neural Machine Translation*
- *Text Summarisation*
- *Learning to Execute*

Problem - for sequence labeling, you may need to incorporate information from both the **preceding** and **following** words. Can RNNs do that?

# Bidirectional RNNs
<center><img src="../img/bilstm.png" width="60%"></center>

- $\overleftarrow{\mathbf{h}}_t$ and $\overrightarrow{\mathbf{h}}_t$ represent (summarise) both the **past** and the **future** around a given sequence element.

# Stacking (Deep LSTMs)

In [7]:
Image(url='../img/deep_bilstm.png'+'?'+str(random.random()), width=800)

# Levels of Granularity

RNNs with tokenization at the character level for:

* Language modeling ([Karpathy, 2015](http://karpathy.github.io/2015/05/21/rnn-effectiveness/))
* Word representation (e.g., [Pinter et al. 2017](https://aclanthology.org/D17-1010/))

RNNs for sentence representation:
* Skip-Thought Vectors ([Kiros et al., 2015](https://proceedings.neurips.cc/paper/2015/file/f442d33fa06832082290ad8544a8da27-Paper.pdf))

In [8]:
Image(url='../img/pinter_char_rnn.png'+'?'+str(random.random()), width=400)

# Summary

* RNNs are *sequence models* that take word embeddings and encode whole sequences
* They can be used as language models with unbounded (but decaying) history window
* Many architectural variations:
    * LSTM
    * BiLSTM
    * Stacked (multi-layer) (Bi)LSTM
* Many possible use cases:
    * One-to-one
    * One-to-many
    * Many-to-many (sequence-to-sequence)

 # Additional Reading #
* [*The Unreasonable Effectiveness of Recurrent Neural Networks.* Blog post by Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [Jurafsky & Martin (see Chapter 9)](https://web.stanford.edu/~jurafsky/slp3/9.pdf)